# Logprobs Analysis with Gemma
This notebook demonstrates how to access and analyze log probabilities from the Gemma 2 model using Hugging Face Transformers.

## Installation
Install necessary libraries.

In [ ]:
!pip install -U torch transformers pandas accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 734.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Authentication
Login to Hugging Face to access the model.

In [47]:
import os
from huggingface_hub import login
from google.colab import userdata

try:
    # Try to retrieve token from Colab secrets
    token = userdata.get('HF_TOKEN')
except ImportError:
    # Fallback to environment variable or manual input
    token = os.environ.get('HF_TOKEN')

if token:
    login(token=token)
else:
    print("HF_TOKEN not found. Please login manually.")
    login()

## Imports
Import required libraries for tensor manipulation and model handling.

In [48]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

## Model Loading
Load the Gemma 2 model and tokenizer. Ensure you have access to the model on Hugging Face.

In [ ]:
model_id = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Helper Functions
Define a function to generate text and capture log probabilities for each token.

In [49]:
def generate_with_logprobs(prompt, max_new_tokens=25):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        return_dict_in_generate=True,
        output_scores=True, # return logprobs too
        do_sample=False     # Greedy decoding for deterministic results
    )

    # The generated sequence (including the prompt)
    generated_ids = outputs.sequences[0]
    # The scores are a tuple of tensors (one tensor per generation step)
    scores = outputs.scores

    # Isolate just the new tokens generated
    generated_tokens = generated_ids[len(inputs.input_ids[0]):]

    results = []
    df_data = []

    # Loop through each generated step
    for i, token_id in enumerate(generated_tokens):
        # Get the logits for this step
        step_logits = scores[i]

        # Apply Softmax to get Log Probabilities
        probs = F.softmax(scores[i][0], dim=-1)
        logprobs = torch.log(probs)

        # Get the log probability of the ACTUAL token chosen
        token_logprob = logprobs[token_id].item()

        # Convert to linear probability (0-100%)
        token_prob = np.exp(token_logprob) * 100

        # Decode the token to text
        token_text = tokenizer.decode(token_id)

        # top 5 other candidates
        top_vals, top_indices = torch.topk(probs, 5)

        other_tokens = [{
            "token": tokenizer.decode(idx),
            "logprob": math.log(v.item()),
            "prob": v.item()*100
        } for v, idx in zip(top_vals, top_indices)]

        top_candidates_str = [f"{tokenizer.decode(idx)} ({v.item()*100:.1f}%)" for v, idx in zip(top_vals, top_indices)]

        token_data = {
            "token": token_text,
            "logprob": token_logprob,
            "prob": token_prob,
            "other_tokens": other_tokens
        }

        results.append(token_data)

        # Create a flat dictionary for the DataFrame
        row_data = token_data.copy()
        row_data["other_tokens"] = ", ".join(top_candidates_str)
        df_data.append(row_data)

    full_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return results, pd.DataFrame(df_data), full_text

In [50]:
def generate_guided_response(prompt, guide, max_new_tokens=200):
    # Combine prompt and guide
    full_input = f"{prompt}\n\n{guide}"

    print(f"--- RESTARTING FROM: ---\n{full_input}\n------------------------\n")

    inputs = tokenizer(full_input, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )

    final_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return final_text

## Experiment 1: Logprobs Analysis
Generate a response to a prompt and analyze the token probabilities.

In [73]:
import json

prompt = "Python code for prime number?"
res, df, full_response = generate_with_logprobs(prompt, 1_000)

with open('data.jsonl', 'a+', encoding='utf-8') as f:
    json_record = json.dumps({
        "prompt": prompt,
        "response": full_response,
        "data": res
    }, ensure_ascii=False)
    f.write(json_record + '\n')

print(full_response)
print("-" * 50)
print("\nToken Analysis:")
df



```python
def is_prime(n):
  """
  Checks if a number is prime.

  Args:
    n: The number to check.

  Returns:
    True if n is prime, False otherwise.
  """
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

# Get user input
num = int(input("Enter a number: "))

# Check if the number is prime
if is_prime(num):
  print(f"{num} is a prime number.")
else:
  print(f"{num} is not a prime number.")
```

**Explanation:**

1. **`is_prime(n)` function:**
   - Takes an integer `n` as input.
   - **Base cases:**
     - If `n` is less than or equal to 1, it's not prime, so return `False`.
   - **Iteration:**
     - Iterates from 2 up to the square root of `n` (inclusive).
     - For each number `i` in this range:
       - If `n` is divisible by `i` (i.e., `n % i == 0`), then `n` is not prime, so return `False`.
   - If the loop completes without finding any divisors, then `n` is prime, so return `True`.

2. **User input:**


,token,logprob,prob,other_tokens
0,\n\n,-0.177409,83.743716,"\n\n (83.7%), (6.9%), \n (5.4%), \n\n\n (1.6..."
1,```,-0.221257,80.151033,"``` (80.2%), ** (6.6%), This (2.0%), Here (1.9..."
2,python,-0.002066,99.793601,"python (99.8%), \n (0.1%), javascript (0.0%), ..."
3,\n,-0.004289,99.571985,"\n (99.6%), (0.3%), \n\n (0.1%), 3 (0.0%), ..."
4,def,-0.076834,92.604351,"def (92.6%), import (6.7%), # (0.6%), from (0...."
...,...,...,...,...
531,.,-0.000096,99.990427,". (100.0%), \n (0.0%), (0.0%), .` (0.0%), ...."
532,\n,-0.160890,85.138530,"\n (85.1%), (14.8%), (0.0%), \n\n (0.0%),..."
533,```,-0.000155,99.984455,"``` (100.0%), ``` (0.0%), ```` (0.0%), (0.0..."
534,\n,-1.161658,31.296691,"\n (31.3%), (19.0%), <end_of_turn> (19.0%), ..."


## Experiment 2: Guided Generation / Continuation
Demonstrate how to continue generation from a specific point or guide the model with a prefix.

In this example, we force the model to start with a Python code block.

In [75]:
prompt = "Python code for prime number?"
guide = """```python
def is_prime(n:"""

guided_response_text = generate_guided_response(prompt, guide, max_new_tokens=1_000)
print(f"--- CONTINUED ANSWER ---\n{guided_response_text}")

--- RESTARTING FROM: ---
Python code for prime number?

```python
def is_prime(n:
------------------------

--- CONTINUED ANSWER ---
Python code for prime number?

```python
def is_prime(n: int) -> bool:
  """
  This function checks if a number is prime.

  Args:
    n: The number to check.

  Returns:
    True if the number is prime, False otherwise.
  """
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

# Get user input
num = int(input("Enter a number: "))

# Check if the number is prime
if is_prime(num):
  print(f"{num} is a prime number.")
else:
  print(f"{num} is not a prime number.")
```

**Explanation:**

1. **Function Definition:**
   - `def is_prime(n: int) -> bool:` defines a function named `is_prime` that takes an integer `n` as input and returns a boolean value (`True` or `False`).

2. **Base Cases:**
   - `if n <= 1:` checks if the number is less than or equal to 1. If it is, it's not prime, so the fun